In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import scipy.stats as stats
import statsmodels.api as sm
import sqlite3

In [10]:
# Create a query to select the necessary data
query = """
SELECT 
    sp.CensusTract, 
    sp.Grade, 
    sp.[Percentage Standard Met and Above], 
    sp.[School Name], 
    ea.CensusTract, 
    ea.[Estimate!!Total:], 
    ea.[Estimate!!Total:!!No schooling completed],
    ea.[Estimate!!Total:!!Nursery school],
    ea.[Estimate!!Total:!!Kindergarten],
    ea.[Estimate!!Total:!!1st grade],
    ea.[Estimate!!Total:!!2nd grade],
    ea.[Estimate!!Total:!!3rd grade],
    ea.[Estimate!!Total:!!4th grade],
    ea.[Estimate!!Total:!!5th grade],
    ea.[Estimate!!Total:!!6th grade],
    ea.[Estimate!!Total:!!7th grade],
    ea.[Estimate!!Total:!!8th grade],
    ea.[Estimate!!Total:!!9th grade],
    ea.[Estimate!!Total:!!10th grade],
    ea.[Estimate!!Total:!!11th grade],
    ea.[Estimate!!Total:!!12th grade, no diploma],
    ea.[Estimate!!Total:!!Regular high school diploma],
    ea.[Estimate!!Total:!!GED or alternative credential],
    ea.[Estimate!!Total:!!Some college, less than 1 year],
    ea.[Estimate!!Total:!!Some college, 1 or more years, no degree],
    ea.[Estimate!!Total:!!Associate's degree],
    ea.[Estimate!!Total:!!Bachelor's degree],
    ea.[Estimate!!Total:!!Master's degree],
    ea.[Estimate!!Total:!!Professional school degree],
    ea.[Estimate!!Total:!!Doctorate degree],
    i.CensusTract, 
    i.[Estimate!!Households!!Total],
    i.[Estimate!!Households!!Total!!Less than $10,000],
    i.[Estimate!!Households!!Total!!$10,000 to $14,999],
    i.[Estimate!!Households!!Total!!$15,000 to $24,999],
    i.[Estimate!!Households!!Total!!$25,000 to $34,999],
    i.[Estimate!!Households!!Total!!$35,000 to $49,999],
    i.[Estimate!!Households!!Total!!$50,000 to $74,999],
    i.[Estimate!!Households!!Total!!$75,000 to $99,999],
    i.[Estimate!!Households!!Total!!$100,000 to $149,999],
    i.[Estimate!!Households!!Total!!$150,000 to $199,999],
    i.[Estimate!!Households!!Total!!$200,000 or more],
    i.[Estimate!!Households!!Median income (dollars)],
    i.[Estimate!!Households!!Mean income (dollars)]
FROM 
    SchoolPerform sp
JOIN 
    EdAttain ea ON sp.CensusTract = ea.CensusTract
JOIN 
    Income i ON sp.CensusTract = i.CensusTract
"""

# Connect to the SQLite database
conn = sqlite3.connect(db_file)

# Load the query results into a DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the DataFrame
df.head()


,CensusTract,Grade,Percentage Standard Met and Above,School Name,CensusTract,Estimate!!Total:,Estimate!!Total:!!No schooling completed,Estimate!!Total:!!Nursery school,Estimate!!Total:!!Kindergarten,Estimate!!Total:!!1st grade,...,"Estimate!!Households!!Total!!$15,000 to $24,999","Estimate!!Households!!Total!!$25,000 to $34,999","Estimate!!Households!!Total!!$35,000 to $49,999","Estimate!!Households!!Total!!$50,000 to $74,999","Estimate!!Households!!Total!!$75,000 to $99,999","Estimate!!Households!!Total!!$100,000 to $149,999","Estimate!!Households!!Total!!$150,000 to $199,999","Estimate!!Households!!Total!!$200,000 or more",Estimate!!Households!!Median income (dollars),Estimate!!Households!!Mean income (dollars)
0,06001401300,6,26.32,Envision Academy for Arts & Technology,06001401300,3541,104,0,0,0,...,2.7,4.3,7.8,5.8,11.2,32.8,8.8,16.7,105556,133185
1,06001401400,6,26.32,Envision Academy for Arts & Technology,06001401400,3655,74,0,0,0,...,3.0,7.5,4.2,8.2,14.8,23.1,6.7,13.6,87208,101569
2,06001401600,6,26.32,Envision Academy for Arts & Technology,06001401600,1777,59,0,0,0,...,5.2,3.5,7.0,11.1,9.6,9.8,10.0,36.4,135234,162072
3,06001402600,6,26.32,Envision Academy for Arts & Technology,06001402600,979,64,0,6,0,...,11.2,0.0,13.2,15.5,6.4,5.7,5.7,13.7,48362,101047
4,06001402700,6,26.32,Envision Academy for Arts & Technology,06001402700,1577,36,0,0,0,...,5.1,4.2,5.4,5.5,8.6,21.4,5.8,22.1,97250,132023
